In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import sys
sys.path.append('../')
sys.path.append('../toolbench/inference')
from toolbench.inference.LLM.tool_llama_model import ToolLLaMA
import argparse
import os
sys.argv = ['']
from toolbench.utils import (
    standardize,
    change_name,
    replace_llama_with_condense
)
from toolbench.inference.utils import SimpleChatIO, generate_stream, react_parser
from toolbench.inference.Downstream_tasks.rapidapi import pipeline_runner, rapidapi_wrapper
import json
import copy
from tqdm import tqdm

from transformers import GenerationConfig
import re

In [3]:
data = json.load(open('data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final.json', 'r'))

In [4]:
source_max_seq_length = 4096
max_seq_length = 8192
ratio = int(max_seq_length/source_max_seq_length)
replace_llama_with_condense(ratio=ratio)
model = ToolLLaMA('data/ToolLLaMA-2-7b-v2', max_sequence_length=max_seq_length)

In [11]:
def parse_model_output(text):
    thought, action, action_input = react_parser(text)
    message = {
        "role": "assistant",
        "content": thought,
        "function_call": {
            "name": action,
            "arguments": action_input
        }
    }
    return message

result = []
for item_idx, item in enumerate(tqdm(data)):
    res_item = copy.deepcopy(item)
    res_item['predictions'] = []
    chain = copy.deepcopy(item['conversations'])
    for step in chain:
        step['role'] = step['from']
        step['content'] = step['value']
        del step['from']
        del step['value']
    for i, step in enumerate(chain):
        if step['role'] == 'assistant':
            thought, action, action_input = react_parser(step['content'])
            if action != 'Finish':
                gt_message = {
                    "role": "assistant",
                    "content": thought,
                    "function_call": {
                        "name": action,
                        "arguments": action_input
                    }
                }
                model.change_messages(chain[:i])
                prompt = model.return_prompt(None, 0)
                pred_messages = model.prediction_non_stream(prompt, num_return_seqs=2, temp=1.5, 
                                                            top_k=100, penalty_alpha=10.0)
                pred_messages = [parse_model_output(message) for message in pred_messages]
                res_item['predictions'].append({'chain_idx': i,
                                                'ground_truth': gt_message,
                                                'prediction': pred_messages}) 
                print(res_item['predictions'][-1])
    result.append(res_item)
    if item_idx > 0 and item_idx % 100 == 0:
        json.dump(result, open('data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors.json', 'w'))       

In [12]:
result

In [6]:
len(result)